<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploratory-Data-Analysis-and-Metric" data-toc-modified-id="Exploratory-Data-Analysis-and-Metric-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploratory Data Analysis and Metric</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span></li><li><span><a href="#Submit" data-toc-modified-id="Submit-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Submit</a></span></li><li><span><a href="#Homework" data-toc-modified-id="Homework-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Homework</a></span></li></ul></div>

In [26]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score, accuracy_score
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import string
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

In [15]:
train_df = pd.read_csv('/home/alexey/MLbase_2021_spring/lecture06/data/train.csv')
test_df = pd.read_csv('/home/alexey/MLbase_2021_spring/lecture06/data/test.csv')

In [16]:
train_df.sentence = \
    train_df.sentence.str.translate(str.maketrans('', '', string.punctuation + string.digits)).str.lower()
test_df.sentence = \
    test_df.sentence.str.translate(str.maketrans('', '', string.punctuation + string.digits)).str.lower()

In [17]:
train_df.drop_duplicates(inplace=True, keep=False)

В релиз не вошло, но в "train.csv" встречается большое количество ошибок, как правило это повторы, как и было анонсированно. Поэтому подчищаем наш трейн вместе с самими дубликатами.

In [18]:
train_df

,sentence,language
0,pensez à la communication le discours les ge...,fr
1,můžete si ji pronajmout vzít na splátky koup...,cs
2,každý starosta pochopil že když mají tyto for...,cs
3,det är ytterligare bevis men ändå — jag kriti...,sv
4,كان الأمر لا يصدق,ar
...,...,...
3159628,そんな所で捕まっている片手は 億の人々と繫がる命綱なのです,ja
3159629,първоначално се опитах да направя думите quot ...,bg
3159630,ho appreso che ha a che fare con lapos attenzi...,it
3159631,e os edifícios não se limitam a apenas evocar ...,pt-br


In [19]:
X_test = test_df.values[:, 1]
label_encoder = LabelEncoder().fit(train_df.values[:, 1])

In [20]:
X_test

array(['תודה לכם ',
       'precisamos de compaixão para começar  e autointeresse sensato para lidar com tudo com seriedade ',
       '這個增長相當大 ， 並且它將引發經濟的增長 。', ...,
       'prečo  lebo je to v každom z nás ',
       'ibm과 usda과 같이 하고 있습니다  이를 저작권 보호가 없는 퍼블릭 도메인에 공개합니다  왜냐하면 마르스는 모든사람이 이 정보에 접근해서 코코아의 생산성이 더 향상되고 지속가능해지기는데 모두가 돕기를 원하기 때문입니다 ',
       '换而言之 ， 我们的恐惧让我们想到未来 。'], dtype=object)

In [21]:
uniq_lan_set = train_df.language.unique()

In [22]:
uniq_lan_set

array(['fr', 'cs', 'sv', 'ar', 'hr', 'id', 'az', 'ru', 'he', 'ja', 'de',
       'nl', 'zh-tw', 'ko', 'it', 'hu', 'en', 'vi', 'bg', 'th', 'zh-cn',
       'tr', 'el', 'nb', 'es', 'pl', 'ro', 'sr', 'fa', 'da', 'uk',
       'pt-br', 'ka', 'zh', 'sl', 'sk', 'lt', 'my', 'mn', 'fr-ca', 'mk',
       'fi', 'sq', 'pt', 'eu', 'hy', 'ta', 'hi', 'et', 'ku', 'gl', 'bn',
       'mr', 'bs', 'be', 'ms', 'eo', 'ur', 'kk'], dtype=object)

<font size=3>Поскольку, как и было анонсированно, классы у нас не сбалансированы, создадим подвыборку размера: 

$n_{min} \times m,$
где $n_{min}$ - количество текстов для самого минорного языка, m - количество языков.
<font>

In [23]:
balanced_size = 2000
X_train = np.array([])
y_train = np.array([])

In [27]:
for lang in tqdm_notebook(uniq_lan_set):
    df_buffer = train_df[train_df.language == lang]
    y_train = np.hstack([y_train, df_buffer.values[:balanced_size, 1]])
    X_train = np.hstack([X_train, df_buffer.values[:balanced_size, 0]])

<ipython-input-27-c1b9805978da>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for lang in tqdm_notebook(uniq_lan_set):


  0%|          | 0/59 [00:00<?, ?it/s]

In [28]:
y_train = label_encoder.transform(y_train)

In [29]:
%%time
pipe = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 3), analyzer='char', min_df=6, max_df=0.6)),
    ('model', SGDClassifier(random_state=42, loss='log'))
])

CPU times: user 352 µs, sys: 0 ns, total: 352 µs
Wall time: 363 µs


In [30]:
# %%time
# pipe = Pipeline([
#     ('vectorizer', TfidfVectorizer(ngram_range=(1, 3), analyzer='char', min_df=6, max_df=0.6)),
#     ('model', MultinomialNB())
# ])

In [31]:
%%time
pipe.fit(X_train, y_train)

CPU times: user 1min 32s, sys: 12.5 s, total: 1min 45s
Wall time: 1min 21s


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer='char', max_df=0.6, min_df=6,
                                 ngram_range=(1, 3))),
                ('model', SGDClassifier(loss='log', random_state=42))])

<font size=3>Ниже реализуется предикт батчами по 50k.

In [32]:
%%time
y_pred = pipe.predict(X_test[0:50000]) 
# Небольшой костыль, вынесено из цикла, 
# тк до релизной версии использовал partial_predict.

CPU times: user 9.65 s, sys: 59.7 ms, total: 9.71 s
Wall time: 9.71 s


In [33]:
%%time
batch_step = 50000
for i in tqdm_notebook(range(batch_step, X_test.shape[0], batch_step)):
    if i >= X_test.shape[0] - batch_step:
        k = X_test.shape[0]
        y_pred = np.hstack([y_pred, pipe.predict(X_test[i:k])])

    else:
        k = i + batch_step
        y_pred = np.hstack([y_pred, pipe.predict(X_test[i:k])])

<timed exec>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/55 [00:00<?, ?it/s]

CPU times: user 8min 57s, sys: 4.01 s, total: 9min 1s
Wall time: 9min 2s


In [34]:
print(y_pred.shape, X_test.shape)

(2784634,) (2784634,)


In [35]:
df = pd.DataFrame({'index': np.arange(X_test.shape[0], dtype=int), 'language': label_encoder.inverse_transform(y_pred)})
df.to_csv('language_detection_submission_4.csv', index=False)